In [2]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from scipy.optimize import minimize
from python_module import blackscholes, sabr
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso

#pd.set_option('display.max_rows', 50)
#pd.set_option('display.max_columns', 50)
#pd.set_option('display.width', 100)

In [3]:
# -------
# BATCH COMPUTATION FUNCTION
# -------
def batch_compute_sabr_option(option_contract_specs_df, F, alpha, beta, rho, nu, r):
    results = dict()
    for i in option_contract_specs_df.index:
        K, T, Qty = option_contract_specs_df.loc[i]
        option_type = 'call' if K >= F else 'put'
        option_symbol = f'{int(T*250)}T {K}K european {option_type}'
        sigma = sabr.compute_vol(F=F, K=K, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu)
        bs_pricing = blackscholes.compute_option(S=F, K=K, T=T, r=r, sigma=sigma, option_type=option_type, compute_greeks=True)
        results[option_symbol] = {'K': K, 'T': T, 'Qty': Qty,'sigma': sigma, 'option_type': option_type, **bs_pricing}
    results_df = pd.DataFrame.from_dict(results, orient='index')
    return results_df


In [4]:
# -------
# OPTION PORTFOLIO SAMPLE
# -------
strikes = np.linspace(start=85, stop=115, num=10, dtype=int)
maturities = np.linspace(start=1/250, stop=60/250, num=10)
strikes, maturities = np.meshgrid(strikes, maturities)
option_contract_specs_df = pd.DataFrame((zip(strikes.ravel(), maturities.ravel())))
option_contract_specs_df.columns = ['K', 'T']

# Add random quantities
np.random.seed(42)
option_contract_specs_df['Qty'] = np.random.normal(size=option_contract_specs_df.shape[0])

# Known parameters
F = 100
beta = 1
r = 0

# Unknown parameters
alpha = 0.2
rho = 0.0
nu = 1

# Create 'market' option portfolio 
option_portfolio_df = batch_compute_sabr_option(option_contract_specs_df=option_contract_specs_df, F=F, alpha=alpha, beta=beta, rho=rho, nu=nu, r=r)

# -------
# FIT SABR MODEL: RECOVER UNKNOWN PARAMETERS
# -------

def objective_function(params, option_contract_specs_df_, market_vol_, F_, r_):
    alpha_, rho_, nu_ = params
    option_portfolio_df_ = batch_compute_sabr_option(option_contract_specs_df=option_contract_specs_df_, F=F_, alpha=alpha_, beta=1, rho=rho_, nu=nu_, r=r_)
    model_vol_ = option_portfolio_df_['sigma'].to_numpy()
    error = np.sum((market_vol_-model_vol_)**2)
    return error

x0 = [0.1, 0, 0.01]
bounds = [(0.001, 10), (-0.99, 0.99), (0.001, 10)]

option_contract_specs_df = option_portfolio_df[['K', 'T', 'Qty']].copy()
market_vol = option_portfolio_df['sigma'].to_numpy()

result = minimize(objective_function, x0, args=(option_contract_specs_df, market_vol, F, r), bounds=bounds)

print("Minimum found at x =", np.round(result.x, 2))
print("Minimum value =", result.fun)
alpha, rho, nu = result.x

Minimum found at x = [0.2 0.  1. ]
Minimum value = 1.577657987269293e-14


In [ ]:
# -------
# COMPUTE OPTION PORTFOLIO SCENARIO PNL
# -------
bump_pnl = dict()
init_pv = option_portfolio_df['price'].to_numpy()
pnl_df = pd.DataFrame(index=option_portfolio_df.index)
for F in np.linspace(start=95, stop=105, num=3):
    for alpha in np.linspace(start=0.1, stop=0.5, num=3):
        for rho in np.linspace(start=-0.9, stop=0.9, num=3):
            for nu in np.linspace(start=0.01, stop=2, num=3):
                option_portfolio_bumped_df = batch_compute_sabr_option(
                    option_contract_specs_df=option_contract_specs_df, 
                    F=F, 
                    alpha=alpha, 
                    beta=beta, 
                    rho=rho, 
                    nu=nu, 
                    r=r)
                pv_bumped = option_portfolio_bumped_df['price'].to_numpy()
                pnl_df.loc[:, f"F {F:.2f}, alpha {alpha:.2f}, rho {rho:.2f}, nu {nu:.2f}"] = pv_bumped - init_pv
y = pnl_df.multiply(option_portfolio_df['Qty'], axis=0).sum()
X = pnl_df.transpose()
y.name = 'y'

In [ ]:
# -------
# GREEDY ALGORITH FOR FEATURE SELECTION
# -------
target_nb_feature = 10
columns = list(X.columns)
nb_iteration = len(columns) - target_nb_feature
model = LinearRegression(fit_intercept=False)
error_results = dict()
for i in tqdm(range(nb_iteration)):
    abs_pnl_error_dict = dict()
    for col in columns:
        X_ = X[columns].drop(col, axis=1)
        model.fit(X_, y)
        y_ = X_.multiply(model.coef_, axis=1).sum(axis=1)
        error = (y-y_).abs().mean()
        abs_pnl_error_dict[col] = error
    abs_pnl_error_s =  pd.Series(abs_pnl_error_dict)
    feature_to_remove = abs_pnl_error_s.idxmin()
    columns.remove(feature_to_remove)
    error_results[len(columns)] = abs_pnl_error_s.min()
X_ = X[columns]
model.fit(X_, y)
y_greedy = X_.multiply(model.coef_, axis=1).sum(axis=1)
y_greedy.name = 'y_greedy'

  0%|          | 0/90 [00:00<?, ?it/s]

In [15]:
# -------
# LASSO REGRESSION
# -------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit Lasso regression
lasso = Lasso(alpha=0.4, max_iter=1000000)
lasso.fit(X_scaled, y)

# Get the coefficients
coefficients = lasso.coef_
coefficients = pd.Series(coefficients)
coefficients.index = X.columns
coefficients = coefficients.replace(0, np.nan).dropna()
print(coefficients.shape[0])

X_ = X[coefficients.index]
model.fit(X_, y)
y_lasso = X_.multiply(model.coef_, axis=1).sum(axis=1)
y_lasso.name = 'y_lasso'

11


In [ ]:
# -------
# PERFORMANCE SUMMARY
# -------
summary = pd.concat([y, y_greedy, y_lasso], axis=1)
(summary['y']-summary['y_greedy']).abs().sum(), (summary['y']-summary['y_lasso']).abs().sum()

(7.5037329673174415, 65.37543531204662)

In [25]:
pd.set_option('display.max_rows', 100)
summary.sort_values('y')

,y,y_greedy,y_lasso
"F 100.00, alpha 0.50, rho 0.90, nu 2.00",-43.280105,-43.160783,-41.613241
"F 100.00, alpha 0.50, rho 0.90, nu 1.00",-38.553357,-38.434507,-37.473100
"F 95.00, alpha 0.50, rho 0.90, nu 2.00",-38.112323,-38.137074,-34.927649
"F 100.00, alpha 0.50, rho 0.00, nu 2.00",-35.524641,-35.659646,-37.678294
"F 105.00, alpha 0.50, rho 0.90, nu 2.00",-34.308227,-34.209132,-34.086496
"F 100.00, alpha 0.50, rho 0.00, nu 1.00",-34.010177,-33.975187,-34.374471
"F 100.00, alpha 0.50, rho 0.90, nu 0.01",-33.545588,-33.447724,-33.324813
"F 100.00, alpha 0.50, rho 0.00, nu 0.01",-33.495196,-33.397740,-33.284131
"F 100.00, alpha 0.50, rho -0.90, nu 0.01",-33.444710,-33.347655,-33.243262
"F 105.00, alpha 0.50, rho 0.90, nu 1.00",-32.623098,-32.553088,-32.010483


In [31]:
list(coefficients.index)

['1T 101.0K european call',
 '27T 98.0K european put',
 '33T 98.0K european put',
 '33T 115.0K european call',
 '40T 98.0K european put',
 '40T 115.0K european call',
 '46T 115.0K european call',
 '53T 95.0K european put',
 '53T 115.0K european call',
 '60T 95.0K european put',
 '60T 115.0K european call']

In [30]:
columns

['1T 95.0K european put',
 '1T 105.0K european call',
 '20T 95.0K european put',
 '20T 108.0K european call',
 '27T 85.0K european put',
 '27T 115.0K european call',
 '33T 91.0K european put',
 '53T 88.0K european put',
 '53T 98.0K european put',
 '53T 108.0K european call']